# Keras Running 6(DIY) 

Keras prvides Earlystopping() which allows us to define where we can stop train our model. 

However, the parameter(**patience**) is a very important parameter. If we only use Earlystopping(), we have to train a net many times to find a good **patience**. It is a waste of time doing this. 

A better way is that we can first train a network for enough epoches or batches. Then we use Earlystopping() to find the best epoch or batch. In this way, we can only train the network once.

Here is the the source codes of EarlyStopping for Keras.GitHub

In [ ]:
class EarlyStopping(Callback):
    '''Stop training when a monitored quantity has stopped improving.
    # Arguments
        monitor: quantity to be monitored.
        patience: number of epochs with no improvement
            after which training will be stopped.
        verbose: verbosity mode.
        mode: one of {auto, min, max}. In 'min' mode,
            training will stop when the quantity
            monitored has stopped decreasing; in 'max'
            mode it will stop when the quantity
            monitored has stopped increasing.
    '''
    def __init__(self, monitor='val_loss', patience=0, verbose=0, mode='auto'):
        super(EarlyStopping, self).__init__()

        self.monitor = monitor
        self.patience = patience
        self.verbose = verbose
        self.patience = 0

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('EarlyStopping mode %s is unknown, '
                          'fallback to auto mode.' % (self.mode),
                          RuntimeWarning)
            mode = 'auto'

        if mode == 'min':
            self.monitor_op = np.less
        elif mode == 'max':
            self.monitor_op = np.greater
        else:
            if 'acc' in self.monitor:
                self.monitor_op = np.greater
            else:
                self.monitor_op = np.less

    def on_train_begin(self, logs={}):
        self.wait = 0       # Allow instances to be re-used
        self.best = np.Inf if self.monitor_op == np.less else -np.Inf

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn('Early stopping requires %s available!' %
                          (self.monitor), RuntimeWarning)

        if self.monitor_op(current, self.best):
            self.best = current
            self.wait = 0
        else:
            if self.wait >= self.patience:
                if self.verbose > 0:
                    print('Epoch %05d: early stopping' % (epoch))
                self.model.stop_training = True
            self.wait += 1

The algorithm is that： 
1. initialize;
2. wait = 0, best = np.Inf
3. get the monitor
4. the main part of this algorithm


My own dxEarlyStopping()

In [ ]:
import numpy as np
def dxEarlyStopping(patience, mode, monitor_log):
    '''arguments:
    patience: the longest iteration wait for
    mode: 'min' or 'max'
    monitor_log: vector of acc of loss
    
    output: we should stop trainning here
    '''
    wait = 0
    if mode == 'min':
            monitor_op = np.less
        elif mode == 'max':
            monitor_op = np.greater
    best = np.Inf if self.monitor_op == np.less else -np.Inf
    
    for i in range(len(monitor_log)):
        current = monitor_log[i]
        if monitor_op(current, self.best):
            best = current
            wait = 0
        else:
            if wait >= patience:
                print('Epoch %05d: early stopping' % (i))
                beststop = i
                break
            self.wait += 1
    return beststop